In [3]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\tuning.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

target_col = 'kilometer_cont'
X = df.drop(target_col, axis=1)
y = df[target_col]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.60, random_state=42)

In [3]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)

print("Gradient Boosting R2:", r2_score(y_test, y_pred_gbr))
print("Gradient Boosting MSE:", mean_squared_error(y_test, y_pred_gbr))

Gradient Boosting R2: 0.5146835549113107
Gradient Boosting MSE: 5397335.975902717


In [10]:
import optuna
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np

# X, y tayyor bo'lgan deb hisoblaymiz
# X_train, X_test, y_train, y_test ham tayyor

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])  # "auto" olib tashlandi
    }

    model = GradientBoostingRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    
    return mse

# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("\n🏆 Best Hyperparameters:")
print(study.best_params)

# Eng yaxshi model bilan qayta trenirovka
best_params = study.best_params
best_model = GradientBoostingRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

print("\nFinal R2:", r2_score(y_test, y_pred))
print("Final MSE:", mean_squared_error(y_test, y_pred))


[I 2025-12-09 09:41:15,830] A new study created in memory with name: no-name-dc1b408f-925a-4ff7-ba82-f97bf6c6d540
[I 2025-12-09 09:43:35,844] Trial 0 finished with value: 5986404.221489255 and parameters: {'n_estimators': 397, 'learning_rate': 0.12452384399441668, 'max_depth': 9, 'subsample': 0.7320450486694876, 'min_samples_split': 12, 'min_samples_leaf': 13, 'max_features': None}. Best is trial 0 with value: 5986404.221489255.
[I 2025-12-09 09:43:56,139] Trial 1 finished with value: 5452359.623146165 and parameters: {'n_estimators': 148, 'learning_rate': 0.15353655112093761, 'max_depth': 4, 'subsample': 0.602191199108453, 'min_samples_split': 9, 'min_samples_leaf': 19, 'max_features': None}. Best is trial 1 with value: 5452359.623146165.
[I 2025-12-09 09:45:00,981] Trial 2 finished with value: 5433617.473722285 and parameters: {'n_estimators': 414, 'learning_rate': 0.11483698154773903, 'max_depth': 3, 'subsample': 0.9419966199453524, 'min_samples_split': 10, 'min_samples_leaf': 10, '


🏆 Best Hyperparameters:
{'n_estimators': 73, 'learning_rate': 0.16042542082969108, 'max_depth': 2, 'subsample': 0.5261435466085267, 'min_samples_split': 6, 'min_samples_leaf': 20, 'max_features': 'log2'}

Final R2: 0.5157481455188659
Final MSE: 5385496.374661272


In [6]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import os
import joblib
import pandas as pd

# Misol uchun ma'lumotlarni o'qib olamiz (siz o'zingizning df bilan almashtiring)
# df = pd.read_csv("your_dataset.csv")

# Ma'lumotlarni belgilash
X = df.drop("kilometer_cont", axis=1)  # target_column o'rniga natija ustunini yozing
y = df["kilometer_cont"]

# Ma'lumotlarni train-test ga bo'lamiz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna dan topilgan eng yaxshi hyperparameterlar bilan modelni yaratamiz
best_params = {
    'n_estimators': 73,
    'learning_rate': 0.16042542082969108,
    'max_depth': 2,
    'subsample': 0.5261435466085267,
    'min_samples_split': 6,
    'min_samples_leaf': 20,
    'max_features': 'log2'
}

model = GradientBoostingRegressor(**best_params, random_state=42)

# Modelni train qilish
model.fit(X_train, y_train)

save_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Models\Ensemble_Models\gradient_boosting_model.joblib"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Natijalarni baholash
y_pred = model.predict(X_test)
print("R2:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

joblib.dump(model, save_path)
print(f"Model muvaffaqiyatli saqlandi: {save_path}")

R2: 0.515194114871532
MSE: 5396236.74022247
Model muvaffaqiyatli saqlandi: C:\Users\Rasulbek907\Desktop\Project_8\Models\Ensemble_Models\gradient_boosting_model.joblib


In [1]:
from IPython.display import display, HTML

# Clickable link (button ko‘rinishida)
display(HTML("""
<a href="error_analysis.ipynb" target="_self" style="
    display: inline-block;
    padding: 10px 20px;
    font-size: 16px;
    color: white;
    background-color: #4CAF50;
    text-align: center;
    text-decoration: none;
    border-radius: 5px;
">
Next Step
</a>
"""))